# Initialization

In [ ]:
import numpy as np
import torch as t
import torchvision.transforms.functional as TF
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import os
import PIL
import cv2
import sys
import time
import imageio as iio
from skimage import filters
from skimage.color import rgb2gray  # only needed for incorrectly saved images
from skimage.measure import regionprops
from skimage.feature import canny
from scipy import ndimage as ndi
import shutil
#from models.res_gru_net import ResidualGRUNet 

## Default parameters 

In [ ]:
depth = 2**8-1.

## Parameters depended on dataset 

In [ ]:
#src = r'D:\data\599'.replace('\\', '/')
src = r'D:\seva\598_processing'.replace('\\', '/')
ido = '598'
idn = ido + 'prc'
dst = src.replace(ido,idn)
k = 120 #how many csv files in a folder, maximum is 120
C_in = 3 #how many views of one seed
D_out = 441

In [ ]:
device = (if t.cuda.is_available())

In [ ]:
if t.cuda.is_available():
    device = "cuda"
else: 
    device = "cpu"
print(device)

## Timelapse printing

In [ ]:
def ntime(s):
    m = 60
    l = [(s/m**2)%24, (s/m)%m, (s)%m]
    return([int(x) for x in l])
def coco(tl, ni, pt):
    sm = sum(tl)
    tt = 10*(pt)
    if sm > tt: 
        l = tuple(ntime(np.mean(tl)*ni-sum(tl)))
    else: 
        return([pt,'',''])
    if sm > tt and pt%15 != 14:
        return([pt+1,str("%02d:%02d:%02d," % l),''])
    elif sm > tt and pt%15 == 14:
        return([pt+1,str("%02d:%02d:%02d," % l),'\n'])

# Training

## Get a batch of images

In [ ]:
c, dc = [0]*2
tip = []
cip = []
for root, directories, filenames in os.walk(r'D:\seva\598prc_processing'.replace('\\', '/')): 
    i = 0
    lc = 0
    for filename in filenames:
        if filename[-3:] == 'csv':
            cip.append(os.path.join(root,filename))
        if filename[-3:] == 'tif':
            tip.append(os.path.join(root,filename))

In [ ]:
len(cip)

In [ ]:
size = 90, 50
imp = r'D:\seva\598prc_processing\1484717\1491988\rotation_000prc.tif'.replace('\\', '/')
im = PIL.Image.open(imp).resize(size, PIL.Image.ANTIALIAS)
im.show()

### getbatch

In [ ]:
bs = 10
bcr = 0
D_out = 441
C_in = 3
def getbatch(bcr):
    label = t.zeros([bs, D_out]).to(device)
    for i in range(bs):
        label[i,:] = labelsb[i+bcr*bs,:].to(device)
    if randinit:
        data = t.rand([bs, C_in, nh, nw]).to(device)
    else:
        data = t.zeros([bs, C_in, nh, nw]).to(device)
        for i in range(bs):
            for j in range(C_in):
                data[i,j,:,:] = transforms.ToTensor()(\
                                PIL.Image.open(tip[C_in*i+j])\
                                .resize([nw,nh], PIL.Image.ANTIALIAS)).unsqueeze_(0)  
    return(data,label)

In [ ]:
labels = np.zeros([len(cip),D_out])
for i in range(len(cip)):
    labels[i,:] = t.tensor(np.genfromtxt(cip[i], delimiter='\n'))

In [ ]:
labelsb = t.tensor((labels - np.mean(labels, axis = 0))/np.std(labels, axis = 0))

In [ ]:
def howbig(ten):
    print("%.2f" % (ten.element_size() * ten.nelement()/1024**3), 'GB')

In [ ]:
howbig(labelsb)

In [ ]:
def getsize(szi,ker,srd): 
    pad = (0,0)
    dil = np.asarray((0,0))
    return(tuple((np.squeeze((szi+2*np.asarray(pad)-dil*[np.asarray(ker)-1]+1)/np.asarray(srd))).astype(int)))

In [ ]:
sz = 32
def szo(sz,ker,srd,pad):
    sz = int(np.floor((sz+2*pad-ker)/srd))+1
    #print(sz)
    return(sz)

In [ ]:
sz = szo(sz,5,1)

## Neural network architecture

In [ ]:
class DNet(t.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = t.nn.Linear(nh*nw*C_in, layers[0])
        self.linear2 = t.nn.Linear(layers[0],layers[1])
        self.linear3 = t.nn.Linear(layers[1],layers[2])
        self.linear4 = t.nn.Linear(layers[2],layers[3])
        self.linear5 = t.nn.Linear(layers[3],layers[4])
        self.do = t.nn.Dropout(p=dop)
    def forward(self, x):
        x = x.view(x.shape[0],-1)
        x = self.do(F.leaky_relu(self.linear1(x).clamp(min=0)))
        x = self.do(F.leaky_relu(self.linear2(x)))
        x = self.do(F.leaky_relu(self.linear3(x)))
        x = self.do(F.leaky_relu(self.linear4(x)))
        x = self.do(self.linear5(x))
        return x
class CNet(t.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv0 = nn.Conv2d(C_in, 8, (5,5))
        self.conv1 = nn.Conv2d(8, 16, (5,5))
        self.pool = nn.MaxPool2d(2,2)
        sz = getsize((nh,nw),(5,5),(1,1))
        sz = getsize(sz,(2,2),(2,2))
        sz = getsize(sz,(5,5),(1,1))
        sz = getsize(sz,2,2)
        sz = np.asarray(sz)-4-int((sc/2)%2)
        self.linear0 = t.nn.Linear(16*sz[0]*sz[1], 1000)
        self.linear1 = t.nn.Linear(1000, D_out)
    def forward(self, x):
        x = self.pool(F.relu(self.conv0(x)))
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(x.shape[0],-1)
        x = F.relu(self.linear0(x))
        x = F.relu(self.linear1(x))
        return x
nnarchitectures = { 'DNet' : DNet, 
                    'CNet' : CNet}

## Training algorithm

In [ ]:
def multit():
#     print('size='+str(nh)+'x'+str(nw)+\
#           ', bn='+str(bn)+\
#             ', bs='+str(bs)+\
#             ', epochs='+str(epochs)+\
#             ', layers='+str(layers)+\
#             ', lr='+str(lr)+\
#             ', noisein='+str(randinit)) 
    starta = time.time()
    #model = None
#   if model is None:
    model = nnarchitectures[modelname]()
    model.to(device)
    optimizer = t.optim.Adam(model.parameters(), lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
    tl = []
    pt = 0
    lossa = np.zeros([2, epochs])
    xval, xtest = [t.zeros([bs, C_in, nh, nw]).to(device)]*2
    yval, ytest = [t.zeros([bs, D_out]).to(device)]*2
    [xtest, ytest] = getbatch(0)
    [xval, yval] = getbatch(1)
    for j in range(epochs):
        start = time.time()  
        for i in range(bn):
            [xtrain, ytrain] = getbatch(i+2)
            try:
                del loss
            except:
                1
            criterion = t.nn.MSELoss(reduction='mean')
            xtrain = xtrain.detach() #xtrain contains images of the circle
            y_pred = model(xtrain) #y_pred consists of coordinates of the circle center and its radius
            if ltype == 'supervised': 
                # Forward pass: Compute predicted y by passing x to the model
                #calculating loss between predicted parameters and parameters which were used for generation
                loss = criterion(y_pred, ytrain)
            else: 
                x_pred = t.zeros([bn,C_in,ht,wt])
                for i in range(bn):
                    # Generating circles with inferred parameters y_pred.
                    x_pred[i,0,:,:] = ellipses(y_pred, k, sz, device)
                # Compute and print loss
                # Compare generated image with the input image, finding per-pixel distance between two images
                loss = criterion(x_pred, xtrain)
            # Zero gradients, perform a backward pass, and update the weights.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        lossa[0][j] = loss.item()
        lossa[1][j] = criterion(model(xval), yval).item()
        del loss
        end = time.time()
        tl.append(end-start)
        col = coco(tl,epochs, pt)
        pt = col[0]
        print(col[1], end = col[2])
    enda = time.time()
    timelapse = enda - starta
    return(model, lossa, timelapse)

## Draw a plot of the training curve

In [ ]:
def traplot():
    ymin, ymax = -4, -2
    plt.rcParams["figure.figsize"] = (18,10)
    linestyles = ['-', '--', '-.', ':', '-', '--', '-.', ':', '-', '--', '-.', ':']
    colors = ['red', 'green']
    axes = plt.gca()
    labels_text = [['train', 'validation'], ['train rand input', 'validation rand input']]
    axes.set_ylim([ymin,ymax])
    for trt in range(2):
        for isr in range(2):
            plt.plot(np.arange(lossb.shape[2]), \
                     np.log(lossb[isr][trt][:]), label=labels_text[trt][isr], \
                     color = colors[isr], linestyle=linestyles[trt], linewidth=3)
    axes.set_xticks(np.arange(0, int(lossb[0][0].shape[0]*1.1),\
                              int(lossb[0][0].shape[0]*0.1)))
    #axes.set_yticks(np.arange(ymin, ymax, 1))
    plt.grid()
    plt.legend(bbox_to_anchor=(1.05, 1), loc='best', borderaxespad=0., fontsize = 24)
    plt.xticks(fontsize=24)
    plt.yticks(fontsize=24)
    #plt.autoscale(enable=True, axis='both', tight=None)
    plt.xlabel('Epoch', fontsize=24)
    plt.ylabel('log(MSE Loss)', fontsize=24)
    plt.savefig('size='+str([nh, nw])+\
            'bn'+str(bn)+\
            'bs'+str(bs)+\
            'epochs'+str(epochs)+\
            'layers'+str(layers)+\
            'lr'+str(lr)+\
            'etime'+format(timelapse,".2f")+\
            'noisein'+str(randinit)+\
                '.png', bbox_inches='tight')
    plt.show()

In [ ]:
#model, lossa, timelapse = multit()

In [ ]:
epochs = 1 #how many epochs 
lr = 5e-5 #learning rate
bn = 1 #how many batches
bs = 1 #how many images in a batch
layers = [4096, 2048, 1024, 750, D_out] #number of neurons at every layer
#layers = [650, 600, D_out] #number of neurons at every layer
#layers = [D_out] #number of neurons at every layer
dop = 0 #drop out percentage 0 to 1. 
#randinit = 1 #0 is ellipse generator, 1 is noise generator
modelname = 'CNet'
ltype = 'supervised'
lossb = np.zeros([2, 2, epochs])
# nh, nw = [5, 9]*10
# sk = 9
# sc = 4*sk+2
sc = 100
nh = 5*sc
nw = 9*sc
t.manual_seed(0)

In [ ]:
randinit = 0
#model = None
t.cuda.empty_cache()
_, lossb[0,:,:], timelapse = multit()

In [ ]:
# for sk in range(0,10,1):
#     sc = 4*sk+3
#     try:
#         model, lossb[0,:,:], timelapse = multit(sc)
#     except:
#         print(sk, end=',')

In [ ]:
# while True:
#     try:
#         x = int(input("Please enter a number: "))
#         break
#     except ValueError:
#         print("Oops!  That was no valid number.  Try again...")

In [ ]:
del model

In [ ]:
for bs in range(50,100,10):
    t.cuda.empty_cache()
    randinit = 0
    model, lossb[0,:,:], timelapse = multit()
    del model
    t.cuda.empty_cache()
    randinit = 1
    model, lossb[1,:,:], timelapse = multit()
    traplot()

In [ ]:
print(lossa.shape)

In [ ]:
traplot()

In [ ]:
model, lossa, timelapse = multit()

In [ ]:
traplot()

In [ ]:
randinit = 0
model, lossb[0,:,:], timelapse = multit()
randinit = 1
model, lossb[1,:,:], timelapse = multit()

In [ ]:
traplot()

## Hyperparameters 

In [ ]:
k = 120

In [ ]:
print(dst)

In [ ]:
epochs = 100 #how many epochs 
lr = 5e-5 #learning rate
bn = 10 #how many batches
bs = 1 #how many images in a batch
#layers = [4096, 2048, 1024, 750, 441] #number of neurons at every layer
#layers = [650, 600, D_out] #number of neurons at every layer
layers = [D_out] #number of neurons at every layer
dop = 0 #drop out percentage 0 to 1. 
randinit = 1 #0 is ellipse generator, 1 is noise generator
modelname = 'DNet'
ltype = 'supervised'
lossb = np.zeros([2, 2, epochs])

In [ ]:
model, lossa, timelapse = multit()

In [ ]:
traplot()

In [ ]:
plt.rcParams["figure.figsize"] = (18,10)
nrows, ncols =1,4
fig = plt.figure()
for i in range(nrows*ncols):
        ax = fig.add_subplot(int(str(nrows)+str(ncols)+str(i+1)), projection='3d') 
        ax.voxels(datac[i+3][0])
        plt.axis('off')
        plt.autoscale(enable=True, axis='both', tight=None)
#dt = str(datetime.fromtimestamp(int(time.time())))
plt.savefig(os.getcwd() + '\plot output'+'\k'+str(k)+'_'+'n'+str(n)+'_' +\
            'time' + time.strftime("%H%M%S") + '.png', bbox_inches='tight')